In [32]:
import torch
import torchvision
from torchvision import transforms,datasets
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader

## Device setting

In [33]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Hyper parametters

In [34]:
num_epochs = 10
batch_size = 100
learning_rate = 0.001
input_size = 28*28
# hidden_size = sqrt(input_size * output_size)
hidden_size = 90
output_size = 10

train_dataset = datasets.MNIST(root="../data", train=True, download=True)
print(train_dataset.data.shape)

MEAN = (train_dataset.data / 255.0).mean()
STD = (train_dataset.data / 255.0).std()

STD, MEAN

torch.Size([60000, 28, 28])


(tensor(0.3081), tensor(0.1307))

## Data loading in

In [35]:
# Using MNIST datasets
composeTransforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(MEAN, STD)
])

train_dataset = datasets.MNIST(root="../data", train=True, download=True,transform=composeTransforms)
test_dataset = datasets.MNIST(root="../data", train=False, download=True,transform=composeTransforms)

# data loader
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


## Setup model

In [36]:
class FFNeuralNet(nn.Module):
    def __init__(self, inputSize, hiddenSize, outputSize):
        super().__init__()
        self.linear1 = nn.Linear(inputSize, hiddenSize)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(hiddenSize, outputSize)
        
    def forward(self, x):
        outputs = self.linear1(x)
        outputs = self.relu(outputs)
        outputs = self.linear2(outputs)
        
        return outputs

## Traning model

In [37]:



model= FFNeuralNet(input_size, hidden_size, output_size).to(device)

# Optimizer and loss
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
n_total_steps = len(train_loader)

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, 28* 28).to(device)
        labels = labels.to(device)
    
        # forward
        outputs = model(images)
        loss = criterion(outputs, labels)
    
        # backward
        optimizer.zero_grad()
        loss.backward()
        # update weights
        optimizer.step()
    
        if( i + 1)%100 == 0 :
            print(f"Epoch {epoch + 1} / {num_epochs} step [{i + 1}/ {n_total_steps}], loss = {loss.item():.4f}")
            

Epoch 1 / 10 step [100/ 600], loss = 0.3896
Epoch 1 / 10 step [200/ 600], loss = 0.2679
Epoch 1 / 10 step [300/ 600], loss = 0.2839
Epoch 1 / 10 step [400/ 600], loss = 0.1982
Epoch 1 / 10 step [500/ 600], loss = 0.1457
Epoch 1 / 10 step [600/ 600], loss = 0.1370
Epoch 2 / 10 step [100/ 600], loss = 0.1808
Epoch 2 / 10 step [200/ 600], loss = 0.1895
Epoch 2 / 10 step [300/ 600], loss = 0.1731
Epoch 2 / 10 step [400/ 600], loss = 0.1380
Epoch 2 / 10 step [500/ 600], loss = 0.1339
Epoch 2 / 10 step [600/ 600], loss = 0.1246
Epoch 3 / 10 step [100/ 600], loss = 0.1026
Epoch 3 / 10 step [200/ 600], loss = 0.0616
Epoch 3 / 10 step [300/ 600], loss = 0.1347
Epoch 3 / 10 step [400/ 600], loss = 0.0875
Epoch 3 / 10 step [500/ 600], loss = 0.1024
Epoch 3 / 10 step [600/ 600], loss = 0.0732
Epoch 4 / 10 step [100/ 600], loss = 0.0362
Epoch 4 / 10 step [200/ 600], loss = 0.0714
Epoch 4 / 10 step [300/ 600], loss = 0.0560
Epoch 4 / 10 step [400/ 600], loss = 0.0347
Epoch 4 / 10 step [500/ 600], lo

## Make prediction

In [38]:

with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for i, (images, labels) in enumerate(test_loader):
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        
        # forward
        outputs = model(images)
        _, outputs = torch.max(outputs, 1)
        
        n_samples += labels.shape[0]
        n_correct += torch.sum(outputs == labels)
    acc = 100.0*n_correct/n_samples
    print(f"Accuracy = {acc} %")
    
    

Accuracy = 97.5999984741211 %


## Saving model

In [39]:
torch.save(model.state_dict(), "mnist_fnn.pth")